In [225]:
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path
import requests
import json
from typing import Tuple

In [226]:
auth_key=''

In [227]:
def get_list_of_districts_ids(project_title: str = 'Rabi 2022-23') -> list[str, ...]:
    url = f"http://45.89.26.151:3001/api/projectsJSON?namep={project_title}"
    headers = {
        'comp_name': 'WP',
        'Authorization': f'Bearer {auth_key}'
    }
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    return [x.get('code_devision') for x in response.json().get('projects')[0].get('atd_units')]

def get_blocks_by_district(district_ids: list[str, ...]) -> pd.DataFrame:
    result_df = pd.DataFrame()
    base_url = f"http://45.89.26.151:3001/api/blocks"
    headers = {
        'comp_name': 'WP',
        'Authorization': f'Bearer {auth_key}'
    }
    payload = {}
    for d_id in district_ids:
        url = '?'.join([base_url, f"idd={d_id}"])
        response = requests.request("GET", url, headers=headers, data=payload)
        if response.status_code != 200:
            print(f"Error: Unable to get blocks by district for {d_id}. {response.content}")
            continue
        if result_df.empty:
            result_df = pd.DataFrame(response.json().get('blocks'))
            continue
        result_df = pd.concat([result_df, pd.DataFrame(response.json().get('blocks'))], ignore_index=True)

    return result_df

def get_coords_of_polygon_center_of_block(block_ids: tuple[str, ...]) -> pd.DataFrame:
  result_df = pd.DataFrame()

  base_url = "http://45.89.26.151:3001/api/polycenter"
  headers = {
        'comp_name': 'WP',
        'Authorization': f'Bearer {auth_key}'
   }
  payload = {}
  for b_id in block_ids:
    url = '?'.join([base_url, f"idb={b_id}"])
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        print(f"Error: Unable to get polygon coords by block id for {b_id}. {response.content}")
        continue
    if result_df.empty:
        result_df = pd.DataFrame(response.json())
        result_df['latitude'] = str(round(float(result_df.loc['lat', 'point']), 5))
        result_df['longitude'] = str(round(float(result_df.loc['lon', 'point']), 5))
        result_df = pd.DataFrame(result_df.loc['lat', :]) \
                        .T.reset_index(drop=True) \
                        [['name', 'latitude', 'longitude']]
        continue
    current_df = pd.DataFrame(response.json())
    current_df['latitude'] = str(round(float(current_df.loc['lat', 'point']), 5))
    current_df['longitude'] = str(round(float(current_df.loc['lon', 'point']), 5))
    current_df = pd.DataFrame(current_df.loc['lat', :]) \
                    .T.reset_index(drop=True) \
                    [['name', 'latitude', 'longitude']]
    result_df = pd.concat([result_df, current_df])



  return result_df.reset_index(drop=True)


def get_history_weather(datamode: str = 'bilinear', src_id: int = 2,
                        geo_points: Tuple[Tuple[float | int, float | int], ...] = ((28.632854, 77.219721),),
                        start_date: str = "1990-01-01", end_date: str = "1990-01-03",
                        out_format: str = 'csv') -> pd.DataFrame | None:
    """
    Parameters
    ----------
    datamode - интерполяция
    src_id - источник данных
    geo_points - набор кортежей широта-долгота географических точек
    start_date - начальная дата
    end_date - конечная дата
    out_format - выходной формат данных: csv, xlsx или загрузка в БД

    Returns
    -------

    """
    weather_hist = pd.DataFrame()
    dir_to_save = Path("out_data")
    dir_to_save.mkdir(exist_ok=True)

    base_url = "http://212.41.22.36:8084/meteo/api/v1/webportal/ptDailyMeteoData"
    parameters = f"srcId={src_id}&dataMode={datamode}&startDate={start_date}&endDate={end_date}"
    url = '?'.join([base_url, parameters])

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {auth_key}'
    }

    # формируем тело запроса из набора координат
    geo_points_dict = list(map(lambda x: dict(latitude=x[0], longitude=x[1]), geo_points))
    geo_points_json = json.dumps(geo_points_dict)

    response = requests.request("POST", url, headers=headers, data=geo_points_json)

    if response.status_code != 200:
        print(f"Error: Unable to fetch meteo data. {response.content}")
        return weather_hist

    weather_hist = pd.DataFrame.from_records(response.json())
    weather_hist = weather_hist.loc[:, ~weather_hist.columns.isin(['sourceId', 'isInterpolated'])]

    if out_format == 'xlsx':
        weather_hist.to_excel(Path(dir_to_save, f"weather_{datetime.now().strftime('%Y%m%d__%H%M%S')}.xlsx"))
        exit()

    elif out_format == 'sql':
        pass
        exit()

    weather_hist.to_csv(Path(dir_to_save, f"weather_{datetime.now().strftime('%Y%m%d__%H%M%S')}.csv"))
    return weather_hist


In [228]:
district_ids = get_list_of_districts_ids()
district_ids

['472',
 '437',
 '110',
 '530',
 '546',
 '160',
 '594',
 '116',
 '388',
 '685',
 '466',
 '488',
 '417',
 '072',
 '087',
 '118',
 '496',
 '523',
 '692',
 '527']

In [229]:
# district_ids = ['472', '437']
blocks_by_district_df = get_blocks_by_district(district_ids)
blocks_by_district_df


Error: Unable to get blocks by district for 530. b'query returned empty result'
Error: Unable to get blocks by district for 546. b'query returned empty result'
Error: Unable to get blocks by district for 160. b'query returned empty result'
Error: Unable to get blocks by district for 594. b'query returned empty result'
Error: Unable to get blocks by district for 388. b'query returned empty result'
Error: Unable to get blocks by district for 685. b'query returned empty result'
Error: Unable to get blocks by district for 417. b'query returned empty result'
Error: Unable to get blocks by district for 118. b'query returned empty result'
Error: Unable to get blocks by district for 496. b'query returned empty result'
Error: Unable to get blocks by district for 523. b'query returned empty result'
Error: Unable to get blocks by district for 692. b'query returned empty result'
Error: Unable to get blocks by district for 527. b'query returned empty result'


,stateCode,state,distrCode,distr,BlockCode,Blockname,BlockAltNames,mrdcode
0,27,Maharashtra,472,Buldhana,4351,Sindkhedraja,None,None
1,27,Maharashtra,472,Buldhana,4343,Khamgaon,None,None
2,27,Maharashtra,472,Buldhana,4345,Malkapur,None,None
3,27,Maharashtra,472,Buldhana,4344,Lonar,None,None
4,27,Maharashtra,472,Buldhana,4340,Chikhli,None,None
...,...,...,...,...,...,...,...,...
82,08,Rajasthan,087,Alwar,585,Thanagazi,None,None
83,08,Rajasthan,087,Alwar,582,Rajgarh,None,None
84,08,Rajasthan,087,Alwar,587,Umren,None,None
85,08,Rajasthan,087,Alwar,586,Tijara,None,None


In [230]:
# blocks = blocks_by_district_df['BlockCode'].to_list()[:2]
blocks = blocks_by_district_df['BlockCode'].to_list()
blocks

['4351',
 '4343',
 '4345',
 '4344',
 '4340',
 '4348',
 '4347',
 '4341',
 '4349',
 '4339',
 '4342',
 '4350',
 '4346',
 '4043',
 '4042',
 '4048',
 '4044',
 '4047',
 '4045',
 '4046',
 '6854',
 '742',
 '740',
 '749',
 '745',
 '743',
 '6859',
 '750',
 '741',
 '7383',
 '747',
 '6857',
 '746',
 '748',
 '744',
 '787',
 '789',
 '790',
 '786',
 '7464',
 '791',
 '788',
 '4280',
 '4285',
 '4290',
 '4278',
 '4283',
 '4277',
 '4288',
 '4287',
 '4281',
 '4279',
 '4286',
 '4289',
 '4284',
 '4282',
 '4497',
 '4496',
 '4499',
 '4494',
 '4500',
 '4495',
 '4501',
 '4498',
 '7358',
 '538',
 '7104',
 '537',
 '539',
 '540',
 '536',
 '575',
 '579',
 '581',
 '576',
 '583',
 '584',
 '7391',
 '580',
 '578',
 '7397',
 '577',
 '585',
 '582',
 '587',
 '586',
 '574']

In [231]:
df_coords = get_coords_of_polygon_center_of_block(blocks)
df_coords

,name,latitude,longitude
0,Sindkhedraja,20.02631,76.29283
1,Khamgaon,20.6113,76.55015
2,Malkapur,20.90061,76.19802
3,Lonar,19.99191,76.50078
4,Chikhli,20.34891,76.3291
...,...,...,...
82,Thanagazi,27.3564,76.28078
83,Rajgarh,27.21384,76.45869
84,Umren,27.57476,76.5615
85,Tijara,27.99186,76.8572


In [232]:
blocks_by_district_df

,stateCode,state,distrCode,distr,BlockCode,Blockname,BlockAltNames,mrdcode
0,27,Maharashtra,472,Buldhana,4351,Sindkhedraja,None,None
1,27,Maharashtra,472,Buldhana,4343,Khamgaon,None,None
2,27,Maharashtra,472,Buldhana,4345,Malkapur,None,None
3,27,Maharashtra,472,Buldhana,4344,Lonar,None,None
4,27,Maharashtra,472,Buldhana,4340,Chikhli,None,None
...,...,...,...,...,...,...,...,...
82,08,Rajasthan,087,Alwar,585,Thanagazi,None,None
83,08,Rajasthan,087,Alwar,582,Rajgarh,None,None
84,08,Rajasthan,087,Alwar,587,Umren,None,None
85,08,Rajasthan,087,Alwar,586,Tijara,None,None


In [233]:
blocks_by_district_df.shape

(87, 8)

In [234]:
df_coords.shape

(87, 3)

In [235]:
geogr_df = blocks_by_district_df.merge(df_coords, left_on='Blockname', right_on='name', how='left')
geogr_df

,stateCode,state,distrCode,distr,BlockCode,Blockname,BlockAltNames,mrdcode,name,latitude,longitude
0,27,Maharashtra,472,Buldhana,4351,Sindkhedraja,None,None,Sindkhedraja,20.02631,76.29283
1,27,Maharashtra,472,Buldhana,4343,Khamgaon,None,None,Khamgaon,20.6113,76.55015
2,27,Maharashtra,472,Buldhana,4345,Malkapur,None,None,Malkapur,20.90061,76.19802
3,27,Maharashtra,472,Buldhana,4344,Lonar,None,None,Lonar,19.99191,76.50078
4,27,Maharashtra,472,Buldhana,4340,Chikhli,None,None,Chikhli,20.34891,76.3291
...,...,...,...,...,...,...,...,...,...,...,...
82,08,Rajasthan,087,Alwar,585,Thanagazi,None,None,Thanagazi,27.3564,76.28078
83,08,Rajasthan,087,Alwar,582,Rajgarh,None,None,Rajgarh,27.21384,76.45869
84,08,Rajasthan,087,Alwar,587,Umren,None,None,Umren,27.57476,76.5615
85,08,Rajasthan,087,Alwar,586,Tijara,None,None,Tijara,27.99186,76.8572


In [236]:
geogr_df = geogr_df[[x for x in geogr_df.columns if x not in ['BlockAltNames', 'mrdcode', 'distrCode', 'stateCode', 'name', 'BlockCode']]]


In [237]:
geogr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 0 to 86
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   state      87 non-null     object
 1   distr      87 non-null     object
 2   Blockname  87 non-null     object
 3   latitude   87 non-null     object
 4   longitude  87 non-null     object
dtypes: object(5)
memory usage: 4.1+ KB


In [238]:
geogr_df

,state,distr,Blockname,latitude,longitude
0,Maharashtra,Buldhana,Sindkhedraja,20.02631,76.29283
1,Maharashtra,Buldhana,Khamgaon,20.6113,76.55015
2,Maharashtra,Buldhana,Malkapur,20.90061,76.19802
3,Maharashtra,Buldhana,Lonar,19.99191,76.50078
4,Maharashtra,Buldhana,Chikhli,20.34891,76.3291
...,...,...,...,...,...
82,Rajasthan,Alwar,Thanagazi,27.3564,76.28078
83,Rajasthan,Alwar,Rajgarh,27.21384,76.45869
84,Rajasthan,Alwar,Umren,27.57476,76.5615
85,Rajasthan,Alwar,Tijara,27.99186,76.8572


In [239]:
tuple((x[0], x[1]) for x in geogr_df[['latitude', 'longitude']].values.tolist())

(('20.02631', '76.29283'),
 ('20.6113', '76.55015'),
 ('20.90061', '76.19802'),
 ('19.99191', '76.50078'),
 ('20.34891', '76.3291'),
 ('20.83265', '76.38383'),
 ('20.67769', '76.1492'),
 ('20.06685', '76.15122'),
 ('21.07964', '76.69768'),
 ('20.4607', '76.11487'),
 ('21.03872', '76.51053'),
 ('20.8108', '76.64345'),
 ('20.25358', '76.61449'),
 ('23.6269', '78.06302'),
 ('23.85002', '78.041'),
 ('23.56824', '77.78321'),
 ('24.08201', '78.0657'),
 ('24.156', '77.71491'),
 ('24.06248', '77.39752'),
 ('23.85716', '77.66573'),
 ('27.07449', '75.09024'),
 ('27.2763', '74.08691'),
 ('26.89334', '74.21529'),
 ('26.81063', '74.63127'),
 ('27.05129', '74.56223'),
 ('27.21726', '74.94194'),
 ('27.30886', '74.71848'),
 ('26.54966', '74.17583'),
 ('27.3305', '74.50169'),
 ('26.62576', '74.39672'),
 ('26.98126', '73.79364'),
 ('27.06793', '73.31491'),
 ('26.65958', '73.91037'),
 ('27.30073', '73.66692'),
 ('27.5284', '74.44077'),
 ('26.35236', '75.35236'),
 ('26.08357', '75.46529'),
 ('26.09831', '

In [240]:
weather_hist_df = get_history_weather(geo_points=tuple((x[0], x[1]) for x in geogr_df.loc[:, ['latitude', 'longitude']].values.tolist()))
weather_hist_df

,latitude,longitude,date,meanTemperature,minimalTemperature,maximalTemperature,precipitation,relativeHumidity,radiation,windSpeed
0,27.27630,74.08691,1990-01-01,11.642393,3.788297,19.846376,0.0,61.490383,15.488771,2.311121
1,27.27630,74.08691,1990-01-02,13.312222,7.687906,20.845390,0.0,53.525925,15.487908,2.987743
2,27.27630,74.08691,1990-01-03,14.636814,9.332658,21.495832,0.0,56.699104,15.383321,3.099378
3,27.28361,77.00659,1990-01-01,10.440390,4.554713,18.017801,0.0,75.572586,15.313934,1.818229
4,27.28361,77.00659,1990-01-02,10.716935,5.699291,16.705940,0.0,74.310920,15.261853,1.382932
...,...,...,...,...,...,...,...,...,...,...
256,26.62576,74.39672,1990-01-02,13.976888,8.677472,20.909170,0.0,49.062737,15.815529,3.281901
257,26.62576,74.39672,1990-01-03,15.161317,10.208823,21.512217,0.0,57.382280,15.566983,3.307881
258,27.98847,76.69921,1990-01-01,9.897883,5.276189,17.009521,0.0,77.757090,14.942708,2.294093
259,27.98847,76.69921,1990-01-02,11.058898,6.092191,16.918028,0.0,71.428170,14.875478,1.068063


In [241]:
geogr_df.loc[:, ['latitude', 'longitude']] = geogr_df.loc[:, ['latitude', 'longitude']].astype(float)
weather_hist_df.dtypes, geogr_df.dtypes
geogr_df

<ipython-input-241-8f01504bef7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geogr_df.loc[:, ['latitude', 'longitude']] = geogr_df.loc[:, ['latitude', 'longitude']].astype(float)


,state,distr,Blockname,latitude,longitude
0,Maharashtra,Buldhana,Sindkhedraja,20.02631,76.29283
1,Maharashtra,Buldhana,Khamgaon,20.6113,76.55015
2,Maharashtra,Buldhana,Malkapur,20.90061,76.19802
3,Maharashtra,Buldhana,Lonar,19.99191,76.50078
4,Maharashtra,Buldhana,Chikhli,20.34891,76.3291
...,...,...,...,...,...
82,Rajasthan,Alwar,Thanagazi,27.3564,76.28078
83,Rajasthan,Alwar,Rajgarh,27.21384,76.45869
84,Rajasthan,Alwar,Umren,27.57476,76.5615
85,Rajasthan,Alwar,Tijara,27.99186,76.8572


In [247]:
final = weather_hist_df.merge(geogr_df, on='latitude', how='left')
final = final.drop(columns=['longitude_y'])
final = final.rename(columns={'longitude_x':'longitude'})
final

,latitude,longitude,date,meanTemperature,minimalTemperature,maximalTemperature,precipitation,relativeHumidity,radiation,windSpeed,state,distr,Blockname
0,27.2763,74.08691,1990-01-01,11.642393,3.788297,19.846376,0.0,61.490383,15.488771,2.311121,Rajasthan,Nagaur,Jayal
1,27.2763,74.08691,1990-01-02,13.312222,7.687906,20.845390,0.0,53.525925,15.487908,2.987743,Rajasthan,Nagaur,Jayal
2,27.2763,74.08691,1990-01-03,14.636814,9.332658,21.495832,0.0,56.699104,15.383321,3.099378,Rajasthan,Nagaur,Jayal
3,27.28361,77.00659,1990-01-01,10.440390,4.554713,18.017801,0.0,75.572586,15.313934,1.818229,Rajasthan,Alwar,Kathumar
4,27.28361,77.00659,1990-01-02,10.716935,5.699291,16.705940,0.0,74.310920,15.261853,1.382932,Rajasthan,Alwar,Kathumar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,26.62576,74.39672,1990-01-02,13.976888,8.677472,20.909170,0.0,49.062737,15.815529,3.281901,Rajasthan,Nagaur,Bherunda
257,26.62576,74.39672,1990-01-03,15.161317,10.208823,21.512217,0.0,57.382280,15.566983,3.307881,Rajasthan,Nagaur,Bherunda
258,27.98847,76.69921,1990-01-01,9.897883,5.276189,17.009521,0.0,77.757090,14.942708,2.294093,Rajasthan,Alwar,Kotkasim
259,27.98847,76.69921,1990-01-02,11.058898,6.092191,16.918028,0.0,71.428170,14.875478,1.068063,Rajasthan,Alwar,Kotkasim


In [243]:
final.shape, final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 260
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   latitude            261 non-null    object 
 1   longitude_x         261 non-null    float64
 2   date                261 non-null    object 
 3   meanTemperature     261 non-null    float64
 4   minimalTemperature  261 non-null    float64
 5   maximalTemperature  261 non-null    float64
 6   precipitation       261 non-null    float64
 7   relativeHumidity    261 non-null    float64
 8   radiation           261 non-null    float64
 9   windSpeed           261 non-null    float64
 10  state               261 non-null    object 
 11  distr               261 non-null    object 
 12  Blockname           261 non-null    object 
 13  longitude_y         261 non-null    object 
dtypes: float64(8), object(6)
memory usage: 30.6+ KB


((261, 14), None)